In [1]:
from langgraph.graph import StateGraph, START, END, add_messages
from typing import TypedDict, Annotated, List
from langgraph.checkpoint.memory import MemorySaver
from langchain_groq import ChatGroq
from langchain_community.tools import TavilySearchResults
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage

In [2]:
memory = MemorySaver()

search_tool = TavilySearchResults(max_results=2)
tools = [search_tool]

llm = ChatGroq(model="llama-3.1-8b-instant")
llm_with_tools = llm.bind_tools(tools=tools)

C:\Users\sunda\AppData\Local\Temp\ipykernel_16108\1180346360.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=2)


In [3]:
class BasicState(TypedDict): 
    messages: Annotated[List, add_messages]

In [4]:
def model(state: BasicState): 
    return {
        "messages": [llm_with_tools.invoke(state["messages"])]
    }

def tools_router(state: BasicState): 
    last_message = state["messages"][-1]
    if(hasattr(last_message, "tool_calls") and 
    len(last_message.tool_calls) > 0):
        return "tools"
    else: 
        return END

In [5]:
graph = StateGraph(BasicState)
graph.add_node(model, "model")
graph.add_node("tools", ToolNode(tools=tools))

graph.set_entry_point("model")
graph.add_conditional_edges("model", tools_router)
graph.add_edge("model", END) 
graph.add_edge("tools", "model")

app = graph.compile(checkpointer=memory, interrupt_before=["tools"])


In [17]:
config = {"configurable": {
    "thread_id": 1
}}

events = app.stream({
    "messages": [HumanMessage(content="which is the smallest mammal?")]
}, config=config, stream_mode="values")

for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

which is the smallest mammal?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (48a022dwx)
 Call ID: 48a022dwx
  Args:
    query: smallest mammal


In [18]:
snapshot = app.get_state(config=config)
snapshot.next

('tools',)

In [19]:
events = app.stream(None, config, stream_mode="values")
for event in events:
    event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (48a022dwx)
 Call ID: 48a022dwx
  Args:
    query: smallest mammal
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "The World's 6 Smallest Mammals | Live Science", "url": "https://www.livescience.com/33049-smallest-mammals.html", "content": "Bumblebee Bat\n\nBumblebee Bat\n\nThe smallest mammal in the world is the bumblebee bat (Craseonycteris thonglongyai), weighing in at just barely 2 grams and measuring 1 to 1.3 inches in length, about the size of a large bumblebee. Discovered by Thai zoologist Kitti Thonglongya in 1974, it's also known as Kitti's hog-nosed bat because of its pink, pig-like snout. [...] The mouse lemur, a species unique to Madagascar, is the world's smallest primate. There are 18 known species of mouse lemur, but on average, they weigh between 1.5 and 3 ounces, accordin